<a href="https://colab.research.google.com/github/arunsrajpurohit/Rainfall-Database/blob/main/Importing_Data_from_IMD_and_Shaping_into_a_usable_database_Then_saving_it_as_CSV_in_google_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imdlib library is of Indian Meteorological Department for accessing data

! pip install imdlib
import numpy as np
import pandas as pd
import imdlib as imd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
start_yr = 1901  # Starting year to download and convert data from
end_yr = 2022  # Ending year
variable = 'rain'  # Variable name (rain for rainfall at 0.25 deg) as we want rainfall data only
file_format = 'yearwise'  # Data for each file will be saved yearly
file_dir = '/content/drive/MyDrive/rainfall_data/rain'  # Path to the directory in Google Drive where CSV file will be saved

error_years = []

try:
    data = imd.open_data(variable, start_yr, end_yr, 'yearwise', file_dir)
    if variable == 'rain':
        grid_size = 0.25
        y_count = 129
        x_count = 135
        x = 66.5
        y = 6.5

    np_array = data.data

    years_no = (end_yr - start_yr) + 1
    day = 0

    for yr in range(0, years_no):
        try:
            filename = file_dir + str(start_yr + yr) + "_" + str(variable) + ".csv"
            with open(filename, 'w') as f:
                if ((start_yr + yr) % 4 == 0) and ((start_yr + yr) % 100 != 0):  # Check for leap year
                    days = 366
                    count = yr + days
                elif ((start_yr + yr) % 4 == 0) and ((start_yr + yr) % 100 == 0) and ((start_yr + yr) % 400 == 0):
                    days = 366
                    count = yr + days
                else:
                    days = 365
                    count = yr + days

                day = day + days

                f.write("X,Y,")
                for d in range(0, days):
                    f.write(str(d + 1))
                    f.write(",")
                f.write("\n")

                for j in range(0, y_count):
                    for i in range(0, x_count):
                        f.write(str((i * grid_size) + x))
                        f.write(",")
                        f.write(str((j * grid_size) + y))
                        f.write(",")
                        time = 0
                        for k in range(day - days, day):
                            val = np_array[k, i, j]
                            if val == 99.9000015258789 or val == -999:
                                f.write(str(-9999))
                                f.write(",")
                            else:
                                f.write(str(val))
                                f.write(",")
                        f.write("\n")
            print("File for " + str(start_yr + yr) + "_" + str(variable) + " is saved")
        except Exception as e:
            print("Error occurred for year " + str(start_yr + yr) + ": " + str(e))
            error_years.append(start_yr + yr)
            continue

    print("CSV conversion successful!")
    if error_years:
        print("Errors occurred in the following years:")
        print(error_years)
except Exception as e:
    print("Error in opening data:", str(e))


Error in opening data: Error in file reading,mismatch in size of data-length
